<a href="https://colab.research.google.com/github/ucguate/climahub-etl/blob/panama/ETL_climahub_PA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Psycopg2 module installation
!pip install psycopg2
!pip install requests

In [ ]:
# importing psycopg2 
import psycopg2 
from psycopg2 import Error
from pprint import pprint
# import requests for rest requests 
import requests
# importing json module
import json
# importing pandas to manage Dictionaries as pandas DataFrames
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# creating querying functions
def show_tables(cursor):
    query = """
      SELECT *
      FROM pg_catalog.pg_tables
      WHERE schemaname != 'pg_catalog' AND 
          schemaname != 'information_schema';
    """
    cursor.execute(query)
    pprint(cursor.fetchall())

def get_table_cols(table_name):
    query = """
    SELECT column_name
      FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name   = '{}'
     ;
    """.format(table_name)

    cursor.execute(query)
    data = cursor.fetchall()
    pprint(data)


In [ ]:
# manipulate data functions
def get_metar(zoom = 18, filter = 'prior', density = 0, taf = 'false', bbox = '-104.94370625,1.7950665583439,-60.998393750001,28.156120443236'):
  queryParams = {
      'zoom': zoom,
      'filter': filter,
      'density': density,
      'taf': taf,
      'bbox': bbox
  }
  response = requests.get("https://927d1d30.us-south.apigw.appdomain.cloud/metar-json", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  header = data["features"][0]
  pprint(header)

  #getting column names
  variables = data["features"][1].keys()
  print(variables)

  df = pd.DataFrame(data["features"][1:], columns = variables)
  display(df)
get_metar()

{'properties': {'bbox': '-104.9437,1.7951,-60.9984,28.1561',
                'beg_time': '1620064660',
                'data': 'METAR',
                'end_time': '1620071860',
                'wrap': ''},
 'type': 'Feature'}
dict_keys(['type', 'id', 'properties', 'geometry'])


,type,id,properties,geometry
0,Feature,410625659,"{'data': 'METAR', 'id': 'KMIA', 'site': 'Miami...","{'type': 'Point', 'coordinates': [-80.317, 25...."
1,Feature,410596229,"{'data': 'METAR', 'id': 'MMMD', 'site': 'Merid...","{'type': 'Point', 'coordinates': [-89.659, 20...."
2,Feature,410604027,"{'data': 'METAR', 'id': 'MMMX', 'site': 'Mexic...","{'type': 'Point', 'coordinates': [-99.072, 19...."
3,Feature,410590815,"{'data': 'METAR', 'id': 'MROC', 'site': 'San J...","{'type': 'Point', 'coordinates': [-84.218, 9.9..."
4,Feature,410594301,"{'data': 'METAR', 'id': 'MWCR', 'site': 'Grand...","{'type': 'Point', 'coordinates': [-81.357, 19...."
...,...,...,...,...
241,Feature,410626587,"{'data': 'METAR', 'id': 'KCLW', 'site': 'Clear...","{'type': 'Point', 'coordinates': [-82.756, 27...."
242,Feature,410626113,"{'data': 'METAR', 'id': 'KNQX', 'site': 'Key W...","{'type': 'Point', 'coordinates': [-81.683, 24...."
243,Feature,410591613,"{'data': 'METAR', 'id': 'KPMP', 'site': 'Pompa...","{'type': 'Point', 'coordinates': [-80.111, 26...."
244,Feature,410620141,"{'data': 'METAR', 'id': 'KRAS', 'site': 'Port ...","{'type': 'Point', 'coordinates': [-97.085, 27...."


# DATABASE CONNECTION and Queries Execution

In [ ]:
connection = None
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="climahub",
                                  password="climahub$2021",
                                  host="138.197.98.178",
                                  port="5432",
                                  database="climahub")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("🟢 CONNECTION SUCCESFUL: PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    show_tables(cursor)
    get_table_cols('station')

except (Exception, Error) as error:
    print("❌ Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

🟢 CONNECTION SUCCESFUL: PostgreSQL server information
{'user': 'climahub', 'dbname': 'climahub', 'host': '138.197.98.178', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

[('public', 'source', 'climahub', None, True, False, True, False),
 ('public', 'station', 'climahub', None, True, False, True, False),
 ('public', 'freq', 'climahub', None, True, False, True, False),
 ('public', 'variable', 'climahub', None, True, False, True, False),
 ('public', 'data', 'climahub', None, True, False, True, False),
 ('public', 'data_last', 'climahub', None, True, False, True, False),
 ('public', 'alert_source', 'climahub', None, True, False, True, False),
 ('public', 'alerts', 'climahub', None, True, False, True, False)]
[('id',),
 ('name',),
 ('type',),
 ('source_